In [15]:
import gdown
import zipfile

url = "https://drive.google.com/uc?id=14gf8bcym_lTcvjZQmg8kwq3aXkENBxMQ"

gdown.download(url=url)

with zipfile.ZipFile("tsukuba.zip", "r") as zip_ref:
    zip_ref.extractall("tsukuba")

Downloading...
From: https://drive.google.com/uc?id=14gf8bcym_lTcvjZQmg8kwq3aXkENBxMQ
To: d:\C\vscode\AIO2024-EXERCISES\Module 2\Final Module\image_depth_estimation\tsukuba.zip
100%|██████████| 354k/354k [00:00<00:00, 2.62MB/s]


Problem 1: Xây dựng hàm tính disparity map của hai ảnh stereo đầu vào (ảnh bên trái (L) và
ảnh bên phải (R)) theo phương thức pixel-wise matching. Các bước tính toán trong phương
pháp này có thể được miêu tả qua các bước dưới đây:
1. Đọc ảnh chụp bên trái (left) và ảnh chụp bên phải (right) dưới dạng ảnh grayscale (ảnh
mức xám) đồng thời ép kiểu ảnh về np.float32.
2. Khởi tạo hai biến height, width có giá trị bằng chiều cao, chiều rộng của ảnh trái.
3. Khởi tạo một ma trận không - zero matrix (depth) với kích thước bằng height, width.
4. Với mỗi pixel tại vị trí (h, w) (duyệt từ trái qua phải, trên xuống dưới) thực hiện các bước
sau:
(a) Tính cost (L1 hoặc L2) giữa các cặp pixel left[h, w] và right[h, w - d] (trong đó
d ∈ [0, disparity_range]). Nếu (w - d) < 0 thì gán giá trị cost = max_cost (max_cost
= 255 nếu dùng L1 hoặc 2552 nếu dùng L2).
(b) Với danh sách cost tính được, chọn giá trị d (doptimal) mà ở đó cho giá trị cost là nhỏ
nhất.
(c) Gán depth[h, w] = doptimal × scale. Trong đó, scale =
255
disparity_range

(Ở Problem 01, các bạn gán mặc định giá trị scale = 16).

In [19]:
import cv2
import numpy as np

In [ ]:
def l1_distance(x, y):
    return abs(x - y)

def l2_distance(x, y):
    return (x - y) ** 2

def pixel_wise_matching_l1(l_img, r_img, dis_range, save_result = True):

    lef = cv2.imread(l_img, 0)
    rig = cv2.imread(r_img, 0)

    left = lef.astype(np.float32)
    right = rig.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_val = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_val

            for j in range(dis_range):
                cost = max_val if (x - j) < 0 else l1_distance(int(left[y, x]), int(right[y, x - j]))
                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result == True:
        print("Saving result...")

        cv2.imwrite("pixel_wise_l1.png", depth)
        cv2.imwrite("pixel_wise_l1_color.png", cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print("Done!")

    return depth

In [21]:
def pixel_wise_matching_l2(l_img, r_img, dis_range, save_result = True):

    lef = cv2.imread(l_img, 0)
    rig = cv2.imread(r_img, 0)

    left = lef.astype(np.float32)
    right = rig.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_val = 255 ** 2

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_val

            for j in range(dis_range):
                cost = max_val if (x - j) < 0 else l2_distance(int(left[y, x]), int(right[y, x - j]))
                
                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result == True:
        print("Saving result...")

        cv2.imwrite("pixel_wise_l2.png", depth)
        cv2.imwrite("pixel_wise_l2_color.png", cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print("Done!")

    return depth

In [22]:
l_img_path = "tsukuba/left.png"
r_img_path = "tsukuba/right.png"
disparity_range = 16

pixel_wise_result_l1 = pixel_wise_matching_l1(
    l_img_path, 
    r_img_path, 
    disparity_range,
    save_result = True
)

Saving result...
Done!


In [23]:
pixel_wise_result_l2 = pixel_wise_matching_l2(
    l_img_path, 
    r_img_path, 
    disparity_range,
    save_result = True
)

Saving result...
Done!
